In [1]:
import matplotlib as plt
import numpy as np
import cv2 as cv
import time

# Using SIFT(Scale Invariant Feature Transform) algorithm in Photogrammetry

## Introduction

In the period 2013 - 2015 I had the chance to work on a project which goal was to create and develop a 3D photogrammetry rig using 54 DSLR cameras. It was built in the shape of a 360°, portable, scanning booth(fig.1) which was used to capture full body scans of people and smaller objects. Later the scans were 3D printed in different scales on a powder, full color 3D printer. 
<img src="./scanning_booth.jpg" style="height:400px">
<p style="text-align: center"><b>fig.1</b></p>

The whole process consisted of four steps:

Firstly, all of the 54 cameras were triggered simultaneously to capture the subject. There are few key moments in this step:
- focal length should be set in the mid-range of the used lens (in our case 18-55mm) in order to introduce minimum non-linear distortions but provide at least 50% overlapping of the images 
- all cameras are calibrated and set to the same shutter speed, aperture and ISO
- the light in the scanning booth must be of sufficient qauntity to provide for:
    - relatively high shutter speed as the people could move during the shoot
    - lower aperture for greater sharpness and dept of field
    - lowest ISO in order to introduce minimum noise in the images
- the light in the scanning boot must be distributed evenly and to be of the same color temperature so the colors of the resulting prints are realistic
- for best results the scanned subjects should: 
    - be wearing clothing with textures and patterns
    - not be wearing too dark and too bright colors
    - not be wearing clothing and accessories with reflective surface

In the second part of the process the images were postprocessed to correct gamma and non-linear lens distorsions, and converted from RAW format into large JPGs to be used as an input of a software called PhotoScan which generated a 3D geometry as an output. The workflow of PhotoScan(as in more of the similar applications) is roughly the following:
- Feature matching across the photos.
At the first stage PhotoScan detects points in the source photos which are stable under viewpoint and lighting variations and generates a descriptor for each point based on its local neighborhood. These descriptors are used later to detect correspondences across the photos. This is similar to the well known SIFT approach, but uses different algorithms for a little bit higher alignment quality.
- Solving for camera intrinsic and extrinsic orientation parameters.
PhotoScan uses a greedy algorithm to find approximate camera locations and refines them later using a bundle-adjustment algorithm.
- Dense surface reconstruction.
At this step several processing algorithms are available. Exact, Smooth and Height-field methods are based on pair-wise depth map computation, while Fast method utilizes a multi-view approach.
- Texture mapping.
At this stage PhotoScan parametrizes a surface possibly cutting it in smaller pieces, and then blends source photos to form a texture atlas.

The third step was to manually refine the generated 3D geometry in an app called zBrush and apply collor corrections on the texture map to get the models ready to be 3D printed.

The last step was printing the models and dipping them first in special solution to improve their strength and color and second in a special wax to make the surface smoother and improve the general outllok.

At the time we started the project the only experience I had was from photography. Most of the development was done by trial and error and by reading articles from few other people who had previously experimented with photogrametry. Now, after taking the course of math concepts and reading some scientific papers I understood big part of the processes and I will try to explain one of the most important ones. All the steps are very important but the quality of the feature matching is the most crucial for the quality of the final accuracy. Any inaccuracy will be compounded until the end influencing all the following processes. I am going to look into the details of the SIFT algorithm. At the end of the article I am going to compare the computation time of the SIFT and SURF algorithms. The SURF algorithm has been developed on the basis of SIFT but with few improvements which allow for much faster computation times with almost no difference in the qaulity of the results.

## SIFT algorithm

**1. Scale-space extrema point detection:** 

The algorithm searches all image scales and locations by computing the Diferences of Gaussians (DoG) for the image with various σ values. The different σ values act like a scale parameter and in this way, feature points that are invariant to scale and rotations are detected. Difference of Gaussians (DoG) is the difference of two blurred versions of the original image. The two blurred versions occur by applying a Gaussian filter with different σ in the original image. Specifically, a DoG image $ D\left(x,y,\sigma \right) $ is given by <br> $ D\left(x,y,\sigma \right)=L\left(x,y,k_{i}\sigma \right)-L\left(x,y,k_{j}\sigma \right) \\ $, <br>
where $ L\left(x,y,k\sigma \right) $ is the convolution of the original image $ I\left(x,y\right) $ with the Gaussian blur $ G\left(x,y,k\sigma \right) $ at scale $ k\sigma $ , i.e.,
$ L\left(x,y,k\sigma \right)=G\left(x,y,k\sigma \right)*I\left(x,y\right) $ <br>
<img src="./DoG.jpg">
<p style="font-size: 12px"><b>fig.2</b> For each octave of scale space, the initial image is repeatedly convolved with Gaussians to produce the set of scale space images shown on the left. Adjacent Gaussian images are subtracted to produce the difference-of-Gaussian images on the right. After each octave, the Gaussian image is down-sampled by a factor of 2, and the process repeated.</p>

Hence a DoG image between scales $ k_{i}\sigma $ and $ k_{j}\sigma $ is just the difference of the Gaussian-blurred images at scales $ k_{i}\sigma $ and $ k_{j}\sigma $. For scale space extrema detection in the SIFT algorithm, the image is first convolved with Gaussian-blurs at different scales. The convolved images are grouped by octave (an octave corresponds to doubling the value of $ \sigma $), and the value of $ k_{i} $ is selected so that we obtain a fixed number of convolved images per octave. Then the Difference-of-Gaussian images are taken from adjacent Gaussian-blurred images per octave.(fig.2)

<img src="./keypoint_selection.jpg">
<p style="text-align: center"><b>fig.3</b></p>

Once DoG images have been obtained, keypoints are identified as local minima/maxima of the DoG images across scales. This is done by comparing each pixel in the DoG images to its eight neighbors at the same scale and nine corresponding neighboring pixels in each of the neighboring scales. If the pixel value is the maximum or minimum among all compared pixels, it is selected as a candidate keypoint.(fig.3)

**2. Keypoint localization:**

Once potential keypoints locations are found, they have to be refined to get more accurate results. Taylor series expansion of scale space is used to get more accurate location of extrema, and if the intensity at this extrema is less than a threshold value of 0.03 the keypoint is discarded.

The DoG function will have strong responses along edges. Therefore, in order to increase stability, we need to eliminate the keypoints that have poorly determined locations but have high edge responses. For this, a 2x2 Hessian matrix is used to compute the principal curvatures. The eigenvalues of the matrix are proportional to the principal curvatures of the point one so the ratio between the two eigen values is calculated. If this ratio is greater than a threshold of 10, that keypoint is discarded. 

So it eliminates any low-contrast keypoints(that are sensitive to noise) and edge keypoints and what remains is strong interest points. In this way, the 2-dimensional translation and scale invariance is reached

**3. Orientation assignment:** 

In this step, each keypoint is assigned one or more orientations based on local image gradient directions. This is the key step in achieving invariance to rotation as the keypoint descriptor can be represented relative to this orientation and therefore achieve invariance to image rotation.

First, the Gaussian-smoothed image $ L\left(x,y,\sigma \right) $ at the keypoint's scale $ \sigma $ is taken so that all computations are performed in a scale-invariant manner. For an image sample $ L\left(x,y\right) $ at scale $ \sigma $, the gradient magnitude, $ m\left(x,y\right) $, and orientation, $ \theta \left(x,y\right) $, are precomputed using pixel differences:

$ m\left(x,y\right)={\sqrt  {\left(L\left(x+1,y\right)-L\left(x-1,y\right)\right)^{2}+\left(L\left(x,y+1\right)-L\left(x,y-1\right)\right)^{2}}} $
$ \theta \left(x,y\right)={\mathrm  {atan2}}\left(L\left(x,y+1\right)-L\left(x,y-1\right),L\left(x+1,y\right)-L\left(x-1,y\right)\right) $ <br>

The magnitude and direction calculations for the gradient are done for every pixel in a neighboring region around the keypoint in the Gaussian-blurred image L. An orientation histogram with 36 bins is formed, with each bin covering 10 degrees. Each sample in the neighboring window added to a histogram bin is weighted by its gradient magnitude and by a Gaussian-weighted circular window with a $ \sigma $ that is 1.5 times that of the scale of the keypoint. The peaks in this histogram correspond to dominant orientations. Once the histogram is filled, the orientations corresponding to the highest peak and local peaks that are within 80% of the highest peaks are assigned to the keypoint. In the case of multiple orientations being assigned, an additional keypoint is created having the same location and scale as the original keypoint for each additional orientation. It contributes to stability of matching.

**4. Keypoint descriptor:**

In the previous two stages, the rotation, scale and 2-dimensional translation invariance was ensured. The goal of this stage is to attain the invariance in illumination and 3-dimensional viewpoint of the features. For this purpose, a local image descriptor incorporates the magnitude of the regional gradient for each feature point at selected scale. A 16x16 neighbourhood around the keypoint is taken. It is devided into 16 sub-blocks of 4x4 size. For each sub-block, 8 bin orientation histogram is created. So a total of 128 bin values are available. It is represented as a vector to form keypoint descriptor. In addition to this, several measures are taken to achieve robustness against illumination changes, rotation etc.

<img src="./descriptor.jpg">

**5. Keypoint Matching**

Keypoints between two images are matched by identifying their nearest neighbours. But in some cases, the second closest-match may be very near to the first. It may happen due to noise or some other reasons. In that case, ratio of closest-distance to second-closest distance is taken. If it is greater than 0.8, they are rejected. It eliminaters around 90% of false matches while discards only 5% correct matches.


## SIFT example and comparison with SURF

In [6]:
img1 = cv.imread('./IMG_1484.jpg')
img2 = cv.imread('./IMG_1485.jpg')

# create Brute Force keypoints Matcher
bf = cv.BFMatcher_create(cv.NORM_L1, crossCheck=False)

# create SIFT instance
sift = cv.xfeatures2d.SIFT_create()

def compare_algorithm_times(surf_extended, surf_upright, run_sift, run_surf):
    # create SURF instance; extended sets 64 or 128 dimentional descriptor and upright is used when 
    # image rotation is less than 10 deg to improve computation speed
    surf = cv.xfeatures2d.SURF_create(extended = surf_extended, upright = surf_upright)

    # genereate keypoints and descriptors with SIFT and measure time
    sift_start_time = time.time()
    kp1_sift, des1_sift = sift.detectAndCompute(img1, None)
    kp2_sift, des2_sift = sift.detectAndCompute(img2, None)
    sift_end_time = time.time()

    # genereate keypoints and descriptors with SURF and measure time
    surf_start_time = time.time()
    kp1_surf, des1_surf = surf.detectAndCompute(img1, None)
    kp2_surf, des2_surf = surf.detectAndCompute(img2, None)
    surf_end_time = time.time()
    
    desc_size = surf.descriptorSize()
    # compare computation time for both algorithms
    print(f"SUFT computation time {surf_end_time - surf_start_time}s with {desc_size}dim descriptor")
    print(F"SIFT computation time {sift_end_time - sift_start_time}s")
    if(run_surf):
        create_SURF_matches(desc_size, kp1_surf, kp2_surf, des1_surf,des2_surf)
    if(run_sift):
        create_SFIT_matches(kp1_sift, kp2_sift, des1_sift, des2_sift)

def create_SFIT_matches(kp1_sift, kp2_sift, des1_sift, des2_sift):
    sift_matches = bf.knnMatch(des1_sift,des2_sift, k=2)
    good_sift_matches = []
    # we take the closest matches and then we show the first 50 for better presentation
    for m,n in sift_matches:
        if m.distance < 0.55*n.distance:
            good_sift_matches.append([m])
    matching_result_sift = cv.drawMatchesKnn(img1, kp1_sift, img2, kp2_sift, good_sift_matches[:50], None, flags=2)
    cv.imwrite("./Matching result_sift.jpg", matching_result_sift)

def create_SURF_matches(desc_size, kp1_surf, kp2_surf, des1_surf, des2_surf):
    surf_matches = bf.knnMatch(des1_surf,des2_surf, k=2)
    good_surf_matches = []
    # we take the closest matches and then we show the first 50 for better presentation
    for m,n in surf_matches:
        if m.distance < 0.55*n.distance:
            good_surf_matches.append([m])
    matching_result_surf = cv.drawMatchesKnn(img1, kp1_surf, img2, kp2_surf, good_surf_matches[:50], None, flags=2)
    cv.imwrite(f"./Matching result_surf_{desc_size}.jpg", matching_result_surf)
          
compare_algorithm_times(False, False, True, True) # 64dim SURF descriptor and no upright optimisation
compare_algorithm_times(True, False, False, True) # 128dim SURF descriptor and no upright optimisation
compare_algorithm_times(False, True, False, False) # 64dim SURF descriptor and upright optimisation
#(run without point matching just for keypoint and descriptors cumputation time comparison)
compare_algorithm_times(False, True, False, False) # 128dim SURF descriptor and upright optimisation
#(run without point matching just for keypoint and descriptors cumputation time comparison)


SUFT computation time 11.650640487670898s with 64dim descriptor
SIFT computation time 14.251821994781494s
SUFT computation time 13.166044235229492s with 128dim descriptor
SIFT computation time 14.725385189056396s
SUFT computation time 8.634482622146606s with 64dim descriptor
SIFT computation time 16.04843544960022s
SUFT computation time 7.139584302902222s with 64dim descriptor
SIFT computation time 15.308313369750977s


<img src="./Matching result_sift.jpg"></img>
<p style="text-align: center">Matching result with SIFT</p>

<img src="./Matching result_surf_64.jpg"></img>
<p style="text-align: center">Matching result SURF with 64dim descriptor</p>

<img src="./Matching result_surf_128.jpg"></img>
<p style="text-align: center">Matching result SURF with 128dim descriptor</p>

## Conclusion

Given SIFT's ability to find distinctive keypoints that are invariant to location, scale and rotation, and robust to affine transformations (changes in scale, rotation, shear, and position) and changes in illumination, they are usable for object recognition. SIFT algorithm is used not only in photogrammetry but also in 2D panorama stitching, robot localization and mapping, augmented reality, object recognition in images and videos, human actions recognition. 

When comparing with the SURF algorithm we can see that in case of 128 dimentioal descriptors and no upright improvement for SURF the computation time is close. Also there is no difference when using 64 dim SURF descriptor and 128 dim SIFT descriptor. I experimented on two different machines and on one of them in this case SIFT was actually faster. When we use upright improvement for SURF it becomes 2 to 3 times faster depending on the machine used, no matter the descriptor's size.

After learning how the feature matching algorithms work I found answers the question I had at the time like: Why people wearing white shirts or black pants at the time of the scan would have their upper or loower body geometry completely missing? Why people wearing jeans and tops with bigger paterns would have perfect geometry but at the same time very small repetitive patterns would not work? Why glasses, wrist watches, phone screens and reflective clothing would not come up properly?
It was also very interesting to read the "Mathematical Foundations of Photogrammetry" paper by Konrad Schindler (see recources p.1) as it gave me a very good overview and intuition about how photogrammetry works and answered more questions about the importance of camera calibration, settings and positioning and images overlaping for the quality of the end result.

## Recources

1. https://www.ethz.ch/content/dam/ethz/special-interest/baug/igp/photogrammetry-remote-sensing-dam/documents/pdf/math-of-photogrammetry.pdf
2. https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf
3. https://en.wikipedia.org/wiki/Scale-invariant_feature_transform 
4. https://www.agisoft.com/forum/index.php?topic=89.0
5. https://en.wikipedia.org/wiki/Difference_of_Gaussians
6. https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_sift_intro/py_sift_intro.html
7. https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_surf_intro/py_surf_intro.html
8. http://orbit.dtu.dk/files/125730941/ISPRS2016GK.pdf